# Import libraries and packages

In [24]:
!ls

new_shootings_new_with_county_dem_and_police_info_2.csv
qiskit-textbook
qiskit-tutorials
QSVM-baseline.ipynb
shootings_new_with_county_dem_and_police_info2.csv
test.csv
Untitled.ipynb


In [25]:
# import qiskit library
import qiskit
import cvxpy # dependency needed for QSVM
from qiskit.tools.monitor import job_monitor

In [26]:
# for plotting
import matplotlib.pyplot as plt
# for matrix and linear algebra
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from qiskit import IBMQ
# IBMQ.save_account(open("token.txt", "r").read())
IBMQ.load_account()
# from qiskit.ml.datasets import ad_hoc_data # preinstalled dataset
# from qiskit import BasicAer # basic simulator
from qiskit.aqua import QuantumInstance # import Quantm Instance to run the experiment
from qiskit.circuit.library import ZZFeatureMap # import quantum feature map
from qiskit.aqua.algorithms import QSVM # import Quantum Support vector Machine model
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name # import several useful unitilies

print("Necessary libraries and packages are imported successfully!")

/opt/conda/lib/python3.8/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '
ibmqfactory.load_account:WARNING:2021-02-26 06:40:30,904: Credentials are already in use. The existing account in the session will be replaced.


Necessary libraries and packages are imported successfully!


# Setup parameters

In [28]:
rawdat = pd.read_csv('new_shootings_new_with_county_dem_and_police_info_2.csv', low_memory=False)
rawdat.shape
rawdat.head()

,Unnamed: 0,State-county FIPS code,county_state,city_state,City,State,% of killed who carried gun,% of killed who were unarmed,% of killed who were race A,% of killed who were race B,...,calc_percentile_police_spending_ratio,calc_misconduct_settlements_per_10k_population,calc_percentile_misconduct_settlements_per_population,calc_police_shootings_per_arrest,calc_percentile_police_shootings_per_arrest,advocacy_tip,civilian_complaints_source,civilian_complaints_source_link,fips_state_adj,fips_county_adj
0,1,1003.0,Baldwin County AL,Bay Minette AL,Bay Minette,AL,1.0,0.0,0.0,0.0,...,6%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0
1,2,1003.0,Baldwin County AL,Baldwin County AL,Baldwin County,AL,0.0,1.0,0.0,0.0,...,6%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0
2,3,1005.0,Barbour County AL,Eufaula AL,Eufaula,AL,1.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5.0
3,4,1009.0,Blount County AL,Brooksville AL,Brooksville,AL,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,9.0
4,5,1015.0,Calhoun County AL,Piedmont AL,Piedmont,AL,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,15.0


In [29]:
rawdat['% of killed who were minority'] = rawdat[["% of killed who were race A",
"% of killed who were race B",
"% of killed who were race H",
"% of killed who were race N",
"% of killed who were race O"]].sum(axis=1)

# Setup data

In [30]:
PREDICTORS = ["use_of_force_complaints_sustained.per.100k", 
              "complaints_in_detention_sustained.per.100k",
              "discrimination_complaints_sustained.per.100k",
              "civilian_complaints_reported.per.100k",
              "calc_low_level_arrests_per_1k_population.per.100k",
              "complaints_in_detention_reported.per.100k",
              "hispanic_as_share_total_arrests"]

In [31]:
PREDICTORS = ["% of killed who carried gun","% of killed who were unarmed",
# "% of killed who were minority",
# "% of killed who had sign of mental illness","% of killed who were male",
# "% of killed who attacked","% of killed whose threat undertermined",
# "% of killed who were fleeing","% of killed whose killer had body camera on",
# "% of killed age 10 and below","% of killed 30 and below","% of killed 50 and below",
# "Estimated percent of people of all ages in poverty 2019",
# "Estimated percent of people age 0-17 in poverty 2019",
# "Estimated percent of related children age 5-17 in families in poverty 2019",
# "Estimate of median household income 2019",
# "POP_ESTIMATE_2019",
# "Unemployment_rate_2019",
"Percent of adults with less than a high school diploma, 2015-19",
"Percent of adults with a high school diploma only, 2015-19",
"Percent of adults completing some college or associate's degree, 2015-19",
"Percent of adults with a bachelor's degree or higher, 2015-19",
"white_population",
"calc_officers_per_10k_population",
"calc_fines_forfeitures_per_resident",
"calc_police_spending_per_resident"]

10

In [32]:
modeldat = rawdat.dropna(subset = ['calc_killed_by_police_per_10k_arrests'] + PREDICTORS,how='any')
modeldat = modeldat.sample(40).reset_index(drop=True)
modeldat.head()

,Unnamed: 0,State-county FIPS code,county_state,city_state,City,State,% of killed who carried gun,% of killed who were unarmed,% of killed who were race A,% of killed who were race B,...,calc_misconduct_settlements_per_10k_population,calc_percentile_misconduct_settlements_per_population,calc_police_shootings_per_arrest,calc_percentile_police_shootings_per_arrest,advocacy_tip,civilian_complaints_source,civilian_complaints_source_link,fips_state_adj,fips_county_adj,% of killed who were minority
0,1085,40089.0,McCurtain County OK,Tom OK,Tom,OK,1.000000,0.000000,0.000000,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,89.0,0.000000
1,1237,47105.0,Loudon County TN,Lenoir City TN,Lenoir City,TN,0.333333,0.333333,0.333333,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.0,105.0,0.333333
2,426,12086.0,Miami-Dade County FL,Little Havana FL,Little Havana,FL,1.000000,0.000000,0.000000,0.0000,...,NaN,NaN,1.4,59%,NaN,NaN,NaN,12.0,86.0,1.000000
3,850,29510.0,St. Louis city MO,St. Louis MO,St. Louis,MO,0.875000,0.031250,0.000000,0.8125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,510.0,0.812500
4,6,1015.0,Calhoun County AL,Eastaboga AL,Eastaboga,AL,1.000000,0.000000,0.000000,1.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,15.0,1.000000


## Convert to a categorical target

In [33]:
modeldat.loc[modeldat['calc_killed_by_police_per_10k_arrests']>=5, 'target'] = 1
modeldat.loc[modeldat['calc_killed_by_police_per_10k_arrests']<5, 'target'] = 0

In [34]:
modeldat = modeldat[['target'] + PREDICTORS]

In [35]:
modeldat['calc_fines_forfeitures_per_resident'] = pd.to_numeric(modeldat['calc_fines_forfeitures_per_resident'].str.replace('$', ''))
modeldat['calc_police_spending_per_resident'] = pd.to_numeric(modeldat['calc_police_spending_per_resident'].str.replace('$', '').str.replace(',', ''))

<ipython-input-35-9dd9f9d1056c>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  modeldat['calc_fines_forfeitures_per_resident'] = pd.to_numeric(modeldat['calc_fines_forfeitures_per_resident'].str.replace('$', ''))
<ipython-input-35-9dd9f9d1056c>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  modeldat['calc_police_spending_per_resident'] = pd.to_numeric(modeldat['calc_police_spending_per_resident'].str.replace('$', '').str.replace(',', ''))


In [54]:
modeldat.shape

(40, 11)

In [36]:
model_train, model_test = train_test_split(modeldat,
                                           test_size=0.33, 
                                           random_state=42)

In [37]:
scaler = MinMaxScaler()
Xtrain_scaled = scaler.fit_transform(model_train[PREDICTORS])
Xtest_scaled = scaler.transform(model_test[PREDICTORS])

In [38]:
train_scaled = pd.DataFrame(Xtrain_scaled)
train_scaled.columns = PREDICTORS
train_scaled['target'] = model_train['target'].reset_index(drop=True)

test_scaled = pd.DataFrame(Xtest_scaled)
test_scaled.columns = PREDICTORS
test_scaled['target'] = model_test['target'].reset_index(drop=True)

## Pre-processing

## Split data and format for QSVM

In [39]:
pos = np.array(train_scaled.loc[train_scaled['target']==1][PREDICTORS])
neg = np.array(train_scaled.loc[train_scaled['target']==0][PREDICTORS])
train={'pos': pos,'neg': neg}

pos = np.array(test_scaled.loc[test_scaled['target']==1][PREDICTORS])
neg = np.array(test_scaled.loc[test_scaled['target']==0][PREDICTORS])
test={'pos': pos,'neg': neg}

In [40]:
datapoints = [scaler.transform(modeldat[PREDICTORS]),
              np.array(modeldat['target'])]

In [41]:
datapoints, class_to_label = split_dataset_to_data_and_labels(train)
print(class_to_label)

{'neg': 0, 'pos': 1}


## Setup the backend to run the QSVM

In [27]:
feature_dim =len(PREDICTORS)
random_seed = 42
shot = 1

In [42]:
# use the qasm simulator
IBMQ.load_account()
provider=IBMQ.get_provider("ibm-q")
backend=provider.get_backend("ibmq_qasm_simulator")
# backend = BasicAer.get_backend('qasm_simulator')
# define the feature map

feature_map = ZZFeatureMap(feature_dim, reps=2)
qsvm = QSVM(feature_map, train, test, None)
qsvm.random_seed = random_seed
quantum_instance = QuantumInstance(backend, shots=shot, seed_simulator=random_seed, seed_transpiler=random_seed)

/opt/conda/lib/python3.8/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '
ibmqfactory.load_account:WARNING:2021-02-26 06:40:37,537: Credentials are already in use. The existing account in the session will be replaced.


In [43]:
result = qsvm.run(quantum_instance)

# Prediction

In [50]:
result['testing_accuracy']

0.8571428571428571

In [47]:
predicted_labels = qsvm.predict(datapoints[0])
predicted_classes = map_label_to_class_name(predicted_labels, qsvm.label_to_class)
print("Ground truth: {}".format(datapoints[1]))
print("Prediction: {}".format(predicted_labels))

# prediction accuracy
print("testing success ratio: {}".format(result['testing_accuracy']))

Ground truth: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1]
Prediction: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1]
testing success ratio: 0.8571428571428571
